In [7]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import os, socket
import numpy as np
import pandas as pd

from CRUD_Python_Module import AnimalShelter

JupyterDash.infer_jupyter_proxy_config()

# connect using your original CRUD (takes no args)
db = AnimalShelter()

# initial load (no projection, drop _id if present)
records = db.read({})
df = pd.DataFrame.from_records(records) if records else pd.DataFrame()
if "_id" in df.columns:
    df = df.drop(columns=["_id"])

# fallback sample row if empty
if df.empty:
    df = pd.DataFrame([{
        "animal_id": "sample-1",
        "name": "Buddy",
        "animal_type": "Dog",
        "breed": "Labrador Retriever Mix",
        "age_upon_outcome_in_weeks": 52,
        "outcome_type": "Adoption",
        "location_lat": 30.27,
        "location_long": -97.74
    }])

# choices
animal_types = sorted(df["animal_type"].dropna().unique()) if "animal_type" in df.columns else []
breeds = sorted(df["breed"].dropna().unique())[:200] if "breed" in df.columns else []
if "age_upon_outcome_in_weeks" in df.columns:
    min_age = int(np.floor(df["age_upon_outcome_in_weeks"].fillna(0).min()))
    max_age = int(np.ceil(df["age_upon_outcome_in_weeks"].fillna(0).max()))
else:
    min_age, max_age = 0, 100
outcomes = sorted(df["outcome_type"].dropna().unique()) if "outcome_type" in df.columns else []

app = JupyterDash(__name__)

# optional logo
image_filename = "grazioso_logo.png"
encoded_image = None
if os.path.exists(image_filename):
    try:
        encoded_image = base64.b64encode(open(image_filename, "rb").read()).decode()
    except Exception:
        encoded_image = None

# controls + reset
controls = html.Div([
    html.Div([
        html.Label("Animal Type"),
        dcc.Dropdown(id="filter-type",
                     options=[{"label": t, "value": t} for t in animal_types],
                     value=None, placeholder="All types", clearable=True)
    ], style={"minWidth": "220px", "marginRight": "1rem"}),

    html.Div([
        html.Label("Breed"),
        dcc.Dropdown(id="filter-breed",
                     options=[{"label": b, "value": b} for b in breeds],
                     value=None, placeholder="All breeds", clearable=True, searchable=True)
    ], style={"minWidth": "260px", "marginRight": "1rem"}),

    html.Div([
        html.Label("Age in Weeks"),
        dcc.RangeSlider(id="filter-age", min=min_age, max=max_age,
                        value=[min_age, max_age], step=1,
                        tooltip={"placement": "bottom", "always_visible": False})
    ], style={"flex": 1, "marginRight": "1rem"}),

    html.Div([
        html.Label("Outcome Type"),
        dcc.Dropdown(id="filter-outcome",
                     options=[{"label": o, "value": o} for o in outcomes],
                     value=None, placeholder="All outcomes", clearable=True)
    ], style={"minWidth": "220px"}),

    html.Button("Reset", id="reset-filters", n_clicks=0, style={"marginLeft": "1rem", "height": "38px"})
], style={"display": "flex", "flexWrap": "wrap", "alignItems": "flex-end", "gap": "0.5rem"})

app.layout = html.Div([
    html.Center([
        html.H1("CS-340 Dashboard — Ryan Davis"),
        (html.Div(html.Img(src=f"data:image/png;base64,{encoded_image}", style={"height": "60px"}))
         if encoded_image else html.Div())
    ]),
    html.Hr(),
    controls,
    html.Hr(),
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "minWidth": "120px", "whiteSpace": "normal", "height": "auto"},
        style_header={"fontWeight": "bold"}
    ),
    html.Br(),
    html.Hr(),
    html.Div(style={"display": "flex", "gap": "1rem"}, children=[
        html.Div(id="graph-id", style={"flex": 1}),
        html.Div(id="map-id", style={"flex": 1})
    ])
])

def _apply_filters(filter_type, filter_breed, age_range, outcome):
    q = {}
    if filter_type:
        q["animal_type"] = filter_type
    if filter_breed:
        q["breed"] = filter_breed
    if outcome:
        q["outcome_type"] = outcome
    age_min, age_max = age_range if age_range else [min_age, max_age]
    q["age_upon_outcome_in_weeks"] = {"$gte": age_min, "$lte": age_max}

    data = db.read(q)  # original CRUD has no projection arg
    dff = pd.DataFrame.from_records(data) if data else pd.DataFrame()
    if "_id" in dff.columns:
        dff = dff.drop(columns=["_id"])
    if dff.empty:
        return pd.DataFrame(columns=df.columns)
    for col in df.columns:
        if col not in dff.columns:
            dff[col] = np.nan
    return dff[df.columns]

@app.callback(
    [Output("filter-type", "value"),
     Output("filter-breed", "value"),
     Output("filter-age", "value"),
     Output("filter-outcome", "value")],
    [Input("reset-filters", "n_clicks")],
    prevent_initial_call=True
)
def reset_filters(n_clicks):
    return None, None, [min_age, max_age], None

@app.callback(Output("datatable-id", "data"),
              [Input("filter-type", "value"),
               Input("filter-breed", "value"),
               Input("filter-age", "value"),
               Input("filter-outcome", "value")])
def update_dashboard(filter_type, filter_breed, age_range, outcome):
    dff = _apply_filters(filter_type, filter_breed, age_range, outcome)
    return dff.to_dict("records")

@app.callback(Output("graph-id", "children"),
              [Input("datatable-id", "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame(viewData) if viewData else df.copy()
    if dff.empty:
        return html.Div("No data to chart")
    top = dff["breed"].fillna("Unknown").value_counts().nlargest(10).reset_index(name="count").rename(columns={"index": "breed"})
    fig = px.pie(top, names="breed", values="count", title="Top 10 Breeds")
    return [dcc.Graph(figure=fig)]

@app.callback(Output("datatable-id", "style_data_conditional"),
              [Input("datatable-id", "selected_columns")])
def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{"if": {"column_id": i}, "background_color": "#D2F3FF"} for i in selected_columns]

@app.callback(Output("map-id", "children"),
              [Input("datatable-id", "derived_virtual_data"),
               Input("datatable-id", "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame(viewData) if viewData else df.copy()
    if dff.empty:
        center = [30.27, -97.74]
        return [dl.Map(style={"width": "100%", "height": "500px"}, center=center, zoom=10,
                       children=[dl.TileLayer(id="base-layer-id")])]
    row = index[0] if index and len(index) > 0 else 0
    row = max(0, min(row, len(dff) - 1))
    lat = dff.iloc[row].get("location_lat", 30.27)
    lon = dff.iloc[row].get("location_long", -97.74)
    breed = dff.iloc[row].get("breed", "Unknown")
    name = dff.iloc[row].get("name", "Animal")
    try:
        center = [float(lat), float(lon)]
    except Exception:
        center = [30.27, -97.74]
    return [dl.Map(style={"width": "100%", "height": "500px"}, center=center, zoom=12, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=center, children=[
            dl.Tooltip(breed),
            dl.Popup([html.H3("Animal Name"), html.P(str(name))])
        ])
    ])]

def _get_free_port():
    s = socket.socket()
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

try:
    JupyterDash._terminate_server_for_port(8050)
except Exception:
    pass

port = 8050
try:
    app.run_server(mode="jupyterlab", port=port, debug=False)
except Exception:
    app.run_server(mode="jupyterlab", port=_get_free_port(), debug=False)


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2025 16:27:37] "GET /_alive_bba51f01-bc69-4342-acb9-005db8f68312 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 16:27:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 16:27:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 16:27:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 16:27:38] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 16:27:38] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [24/Oct/2025 16:27:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 16:27:38] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 16:27:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 16:27:38] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [